# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Loan Data once again - this time the strutured data available through the CSV!

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

loader = CSVLoader(
    file_path=f"./data/complaints.csv",
    metadata_columns=[
      "Date received", 
      "Product", 
      "Sub-product", 
      "Issue", 
      "Sub-issue", 
      "Consumer complaint narrative", 
      "Company public response", 
      "Company", 
      "State", 
      "ZIP code", 
      "Tags", 
      "Consumer consent provided?", 
      "Submitted via", 
      "Date sent to company", 
      "Company response to consumer", 
      "Timely response?", 
      "Consumer disputed?", 
      "Complaint ID"
    ]
)

loan_complaint_data = loader.load()

for doc in loan_complaint_data:
    doc.page_content = doc.metadata["Consumer complaint narrative"]

Let's look at an example document to see if everything worked as expected!

In [5]:
loan_complaint_data[0]

Document(metadata={'source': './data/complaints.csv', 'row': 0, 'Date received': '03/27/25', 'Product': 'Student loan', 'Sub-product': 'Federal student loan servicing', 'Issue': 'Dealing with your lender or servicer', 'Sub-issue': 'Trouble with how payments are being handled', 'Consumer complaint narrative': "The federal student loan COVID-19 forbearance program ended in XX/XX/XXXX. However, payments were not re-amortized on my federal student loans currently serviced by Nelnet until very recently. The new payment amount that is effective starting with the XX/XX/XXXX payment will nearly double my payment from {$180.00} per month to {$360.00} per month. I'm fortunate that my current financial position allows me to be able to handle the increased payment amount, but I am sure there are likely many borrowers who are not in the same position. The re-amortization should have occurred once the forbearance ended to reduce the impact to borrowers.", 'Company public response': 'None', 'Company'

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "LoanComplaints".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [6]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    loan_complaint_data,
    embeddings,
    location=":memory:",
    collection_name="LoanComplaints"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [7]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [8]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [9]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [10]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [11]:
naive_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided context, the most common issues with loans appear to be related to mismanagement and problems with the information and handling by loan servicers. Specifically, common issues include errors in loan balances, misapplied payments, wrongful denials of payment plans, incorrect reporting of account status (such as being marked delinquent without cause), and difficulty applying payments appropriately. There are also recurring complaints about loan transfers without proper notification, discrepancies in loan balances, and issues with handling of payment plans and forgiveness.\n\nIn summary, the most common issue with loans highlighted in the complaints is mishandling or mismanagement by loan servicers, leading to errors in balances, payment application, information reporting, and lack of transparency.'

In [12]:
naive_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints, yes, some complaints indicate that they were not handled in a timely manner. Specifically, the complaint with Complaint ID 12709087 from MOHELA was marked as "Timely response?": No, indicating it was not handled promptly. The complaint describes delays in processing, with unmet expectations of response times. \n\nAdditionally, other complaints mention ongoing issues and delays, such as the complaint with Complaint ID 12973003 from EdFinancial Services, where the issue remains unresolved over 2-3 weeks despite promises, and the complaint with Complaint ID 12975634, which has been open for nearly 18 months without resolution.\n\nTherefore, yes, there were complaints that did not get handled in a timely manner.'

In [13]:
naive_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People often fail to pay back their loans due to several interconnected reasons highlighted in the complaints:\n\n1. **Lack of Clear Information and Communication:** Borrowers report being inadequately informed about their loan status, payment resumption dates, transfer of loans between servicers, or changes in payment requirements. For example, some complaints mention not being notified about loan transfers or the start of repayment, leading to surprise delinquencies.\n\n2. **Compounding and Accumulating Interest:** Many borrowers describe how interest continues to accrue during forbearance or deferment periods, sometimes negating any payments made. This makes it difficult to pay down the actual principal and extends the repayment timeline, discouraging borrowers.\n\n3. **Difficulty Affording Payments:** A common theme is that borrowers cannot increase monthly payments without sacrificing essential living expenses like food, rent, or transportation. The repayment plans often do not a

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [14]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(loan_complaint_data, )

We'll construct the same chain - only changing the retriever.

In [15]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [16]:
bm25_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided data, the most common issue with loans appears to be problems related to dealing with lenders or servicers. This includes issues such as incorrect information about the loan, difficulty in applying payments correctly, lack of transparency, and failure to provide accurate explanations or responses. Several complaints highlight frustrations with how payments are handled, incorrect or confusing information, and perceived unfair or predatory practices by loan servicers.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided information, all the complaints mentioned were responded to with a "Closed with explanation" status and "Yes" for timely response. There is no indication of any complaints that did not get handled in a timely manner.'

In [18]:
bm25_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for various reasons, including issues with the loan servicing process such as being steered into the wrong types of forbearances, lack of communication from the loan servicers about repayment plans or account status, technical problems like reversed payments and incorrect billing, and the transfer of loans to new companies without proper notification. Additionally, some borrowers experienced confusion or deception regarding their repayment obligations and autopay setups, leading to missed payments or negative impacts on their credit scores. In some cases, borrowers felt they did not receive adequate support or information, making it difficult to stay current with their loan payments.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

#### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.


*(A very specific Oracle database error message.)*

---

#### Why BM25 is the better retriever here

| Factor | BM25 | Dense embeddings |
|--------|------|-----------------|
| **Exact‐token overlap** | Scores the document that contains the *exact* string “ORA‑01722” (plus the colon and phrase) highest, because all rare tokens match. | Sub‑word tokenisers split “ORA‑01722” into fragments like `["ORA", "-", "017", "22"]`; those pieces carry little semantic meaning, so the vector is not strongly distinctive. |
| **Rarity of terms** | The error code is extremely rare in the corpus → BM25’s *inverse‑document‑frequency* boost makes any doc that contains it shoot to the top. | Embeddings average over token vectors; rare or OOV tokens often get mapped to generic “unknown” space, diluting the signal. |
| **Need for verbatim context** | Troubleshooting requires the *exact* error text to locate the relevant fix or SQL snippet. | A semantic match like “numeric conversion failed” might be *similar* but could surface docs for a **different** Oracle error, leading to wrong instructions. |
| **Zero‑shot generalisation not helpful** | The user isn’t asking for conceptual advice—they need the precise solution that matches the code. | Embeddings shine when paraphrase or broader meaning matters, not for pinpoint code/look‑up queries. |

**Result**  
With BM25, the top result is almost certainly the official Oracle docs or a Stack Overflow post whose title begins with *“ORA‑01722: invalid number – how to fix”*. Embedding‑only retrieval often buries that exact match behind semantically “similar” errors, making the user scroll or fail to retrieve the right chunk.

*In short, when the query hinges on a rare, literal token (error codes, part numbers, legal citations, etc.), lexical BM25 retrieval outperforms purely semantic vectors.*


## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [19]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [20]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [21]:
contextual_compression_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided information, the most common issue with loans appears to be problems related to the handling and management of the loans by servicers. This includes errors in loan balances, misapplied payments, wrongful denials of payment plans, inaccurate or incomplete information, incorrect interest calculations, data mishandling, and lack of proper communication and documentation. These issues often lead to confusion, credit reporting discrepancies, and disputes over the status and amount owed on the loans.'

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the information provided, it appears that at least one complaint was handled in a timely manner. Specifically, the complaint regarding issues with auto pay and bank account missing was responded to with a "Closed with explanation" status, and the response was marked as "Yes" for timely response. \n\nHowever, there are also complaints indicating delays or ongoing unresolved issues, such as the case where a request to review and respond for nearly 18 months has not been resolved yet. \n\nOverall, while some complaints were managed promptly, others remained unresolved for extended periods. Therefore, yes, some complaints did not get handled in a timely manner.'

In [23]:
contextual_compression_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'Based on the provided context, people failed to pay back their loans primarily due to a combination of factors including lack of clear communication and understanding about their loan obligations, the accumulation of interest during periods of forbearance or deferment, and difficulties in affording payments given their financial circumstances. Specifically, some borrowers were not adequately informed about the necessity of repayment, how interest would accrue and compound, or the impact of their repayment choices, which led to increasing balances despite payments. Additionally, many borrowers found that the available repayment options, such as forbearance or deferment, resulted in growing interest and extended repayment periods, making it harder to pay off the loans. Hardships were further exacerbated by unstable employment, stagnant wages, and misconceptions about loan forgiveness programs, all contributing to the inability to successfully repay the loans.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [24]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [25]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [26]:
multi_query_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'The most common issue with loans, based on the complaints provided, appears to be problems related to servicing and handling of the loans. These include:\n\n- Trouble with how payments are being handled, including issues applying additional funds, misapplied payments, and difficulties in managing repayment plans.\n- Inaccuracies or discrepancies in loan balances, interest calculations, and reporting to credit bureaus.\n- Lack of transparency or communication from loan servicers regarding account status, loan transfers, or changes in terms.\n- Problems with loan classification or verification, such as misclassification of loan types or issues with deferments and in-school status.\n- Errors in credit reporting, leading to significant drops in credit scores without proper notification or opportunity to correct.\n- Challenges in obtaining loan information, documentation, or validation.\n- Servicer misconduct, including not providing proper notice of loan resumption, mishandling forbearanc

In [27]:
multi_query_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Yes, based on the provided complaints, several complaints indicate that issues were not handled in a timely manner. For example:\n\n- One complaint from EdFinancial Services (row 70) reported being waiting over 7 days for a callback after being promised one, with no follow-up received.\n- Another from Nelnet (row 66) involved an unresolved dispute that was not addressed within the required timeframe, with attempts to correct credit reporting issues also delayed.\n- Multiple complaints mention delays of over a year in responses or corrections, such as the complaint about a complaint submitted over 18 months ago still not being resolved or addressed.\n\nOverall, there are multiple instances where complaints were not addressed promptly, some taking several months or over a year without resolution.'

In [28]:
multi_query_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

"People failed to pay back their loans for various reasons, including:\n\n1. Errors and misconduct by student loan servicers, such as misapplied payments, errors in loan balances, wrongful denials of payment plans, and misconduct during collection efforts.\n2. Lack of proper communication or notification about loan status, payments, or transfers, leading to unawareness of due dates or delinquency status.\n3. Poor management or mishandling of loan information, including incorrect reporting to credit bureaus and difficulty in obtaining accurate account information.\n4. Complicated or confusing payment options, such as forbearance or consolidation, often leading to unintentional default or interest capitalization, which increases the debt.\n5. Enforcement of practices like forbearance steering, coercing borrowers into long-term forbearances or consolidations without informing them of better options like income-driven repayment plans.\n6. Technical and administrative issues, such as missin

#### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

### Why *multiple reformulations* lift recall in a RAG pipeline

1. **Covers lexical mismatch**  
   - A single phrasing may use words that never appear in the corpus.  
   - Paraphrases swap in synonyms or domain‑specific jargon, so at least one version shares tokens with the relevant chunk → higher chance of a BM25 / hybrid hit.

2. **Hits diverse semantic neighborhoods**  
   - Dense‑vector models embed each reformulation at a slightly different spot in embedding space.  
   - When you query with all of them and **union** the top‑k results, you sweep a broader radius around the user’s intent, surfacing documents that any one vector might have missed.

3. **Balances granularity**  
   - Long, detailed questions can be broken into *atomic sub‑queries* (“Who?”, “When?”, “Why?”).  
   - Short, ambiguous questions can be expanded into richer versions that add context.  
   Both moves retrieve evidence that matches either the coarse or fine aspects of the user’s need.

4. **Mitigates mistakes in user wording**  
   - Typos, wrong acronyms, or partial product names in the original query get corrected in some reformulations, rescuing what would otherwise be a miss.

5. **Reduces recall variance across corpora**  
   - Different documents describe the same concept in different ways (e.g., “heart attack”, “myocardial infarction”). Multiple rewrites align with multiple author styles.

---

#### Quick example

| Variant | What it targets |
|---------|-----------------|
| “**best iPhone battery life**” | conversational blogs / reviews |
| “**iPhone with highest mAh capacity**” | spec sheets, tech forums |
| “**longest‑lasting iPhone model in 2025**” | news coverage & benchmarks |

Unioning results from all three queries captures both marketing copy and engineering data—boosting recall compared to any single phrasing.

> **Rule of thumb:** More *diverse* reformulations (different vocab, length, focus) → more recall gains; but you still cap the total retrieved passages to stay within the context window.


## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [34]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = loan_complaint_data
child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [35]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [36]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [37]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [38]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [39]:
parent_document_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'The most common issue with loans, based on the complaints provided, appears to be related to errors and misconduct in federal student loan servicing. Specific recurring problems include incorrect information on credit reports, misapplication of payments, wrongful denials of payment plans, discrepancies in loan balances and interest rates, and issues with collection and verification of debts.'

In [40]:
parent_document_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided information, it appears that several complaints were not handled in a timely manner. Specifically, the complaints related to the student loan issues with MOHELA (Complaint IDs 12709087 and 12935889) indicate that the responses were "No" in the "Timely response?" field, meaning they were not handled promptly. Additionally, the complaint about the dispute settlement with Nelnet (Complaint ID 13205525) was responded to within the expected timeframe ("Yes" in "Timely response?"). \n\nTherefore, yes, some complaints—particularly those regarding MOHELA—did not get handled in a timely manner.'

In [41]:
parent_document_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for various reasons, including:\n\n1. Lack of proper communication or notification from loan servicers about payment obligations, as indicated by complaints about not being notified when payments were due or about changes in loan ownership.\n2. Financial hardship or severe economic difficulties that made it impossible to make timely payments, such as unemployment or inability to find employment in their field.\n3. Misrepresentation or lack of transparency from educational institutions and loan providers regarding the long-term financial consequences, job prospects after graduation, and the sustainability of the school’s operations.\n4. Relying on deferment and forbearance options that increased interest and debt over time.\n5. Disputes over the legitimacy or ownership of the debt, including issues related to the legal verification of loans and deceptive practices by collection agencies.\n6. Personal health issues or other personal circumstances th

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [42]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [43]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [44]:
ensemble_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'The most common issue with loans, based on the provided data, appears to be dealing with the loan servicer or lender, including errors in loan balances, misapplied payments, wrongful denials of payment plans, and problems with how payments are being handled. Several complaints highlight issues such as receiving bad information about loans, inability to properly apply payments to principal, inaccurate reporting of delinquency, and mishandling of loan transfers or consolidations. \n\nIn summary, a predominant and recurring problem is the mismanagement and poor communication from loan servicers, which leads to misapplied payments, incorrect account information, and difficulties in resolving repayment issues.'

In [45]:
ensemble_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints, yes, there are several instances indicating complaints not handled in a timely manner. For example:\n\n- One complaint (#12935889) about Mohela was marked as "Timely response?": No.\n- Another (#12744910) regarding inaccuracies in reporting and an ongoing dispute was "Timely response?": Yes, but the complaint was about inaccurate reporting and delays in correction, suggesting the issue persisted over time.\n- Multiple complaints (#12739706, #13062402, #13126709, #13127090, and others) mention delays, extended wait times, or responses that were not addressed promptly, with some even explicitly stating they did not receive responses within expected timeframes.\n- There are cases where the response was "Closed with explanation" but the delays or unresolved issues strongly imply they were not handled promptly or adequately.\n\nOverall, the evidence suggests that at least some complaints were not handled in a timely manner, as indicated directly by the res

In [46]:
ensemble_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for several reasons, often related to mismanagement, misinformation, and systemic issues. Based on the provided complaints, common reasons include:\n\n1. **Lack of Notification and Communication:** Many borrowers were not properly notified about loan transfers, due dates, or repayment start dates, leading to unintentional delinquency and missed payments.\n\n2. **Misleading or Incomplete Information:** Borrowers reported receiving incorrect or misleading information about their loan balances, repayment obligations, or eligibility for programs like income-driven repayment or forgiveness, which caused confusion and unintended default.\n\n3. **System Errors and Technical Difficulties:** Issues such as online portal lockouts, incorrect account statuses, and errors in reporting contributed to borrowers not making payments or being marked delinquent improperly.\n\n4. **Inadequate Support and Assistance:** Borrowers often found customer service unhelpful,

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [47]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [49]:
semantic_documents = semantic_chunker.split_documents(loan_complaint_data[:20])

Let's create a new vector store.

In [50]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="Loan_Complaint_Data_Semantic_Chunks"
)

We'll use naive retrieval for this example.

In [51]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [52]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [56]:
semantic_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided complaints, the most common issues with loans appear to be related to difficulties in communication and account management, such as:\n\n- Struggling to repay loans due to errors or issues with payment plans.\n- Problems with loan reporting, including incorrect or improper reporting of account status or default.\n- Difficulties in obtaining clear information about loan balances, loan servicer changes, or payment amounts.\n- Issues with loan servicing companies failing to respond appropriately or failing to verify or process applications.\n- Unauthorized or illegal reporting and collection practices, including violations of privacy laws.\n\nWhile these are specific to student loans in the context provided, a recurring theme is that many complaints involve mismanagement, lack of transparency, or errors in the handling of loans and related information. \n\nTherefore, a common underlying issue with loans, especially highlighted here, is **mismanagement or errors in se

In [57]:
semantic_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints, it appears that many complaints were responded to in a timely manner, with responses marked as \'Yes\' under the \'Timely response?\' field. Notably, several complaints state "Closed with explanation," indicating that they were addressed within the required time frame. \n\nHowever, there is at least one complaint regarding a lack of response or handling—specifically, the complaint about Nelnet (row 17). The consumer\'s narrative details multiple issues with lack of responses and conduct that suggests their complaint was not handled promptly or satisfactorily.\n\nIn summary:\n\n- Multiple complaints confirm responses were handled in a timely manner.\n- One complaint (about Nelnet\'s failure to respond to Certified Mail and ongoing misconduct) indicates that the complaint was not properly handled or responded to, suggesting that some complaints did not get handled in a timely manner.\n\nTherefore, yes, some complaints did not get handled in a timely man

In [58]:
semantic_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

"People failed to pay back their loans for various reasons, including issues such as difficulties dealing with their loan servicers, miscommunications or inadequate information about their loan status, problems with payment processing, and disputes over the legitimacy or accuracy of their loan details. Some specific reasons noted in the complaints include receiving bad information about loan statuses, delays or errors in re-amortizing payments after forbearance ended, and inaccurate reports of default or delinquency. Additionally, instances of alleged mismanagement, lack of transparency, or improper handling of personal data have also contributed to borrowers' difficulties in repayment."

#### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [ ]:
### YOUR CODE HERE